In [2]:
# ==========================================
# [실습] 멀티 에이전트 협업 아키텍처 비교: 비밀 금고를 열어라!
# ==========================================

#
# 이 실습에서는 동일한 문제(비밀 금고 해제)를 해결하기 위해
# 두 가지 다른 멀티 에이전트 조직 구조(Organization Architecture)를 구현하고 비교합니다.
#
# 1. Shared Memory (Blackboard) 패턴: 리더 없이 자율적인 에이전트들이 게시판을 공유하며 협업
# 2. Hierarchical (Centralized) 패턴: 매니저 에이전트가 계획을 세우고 작업자에게 지시
#
# 준비물: OpenAI API Key가 필요합니다.
# ==========================================

In [3]:
# 1. 라이브러리 설치 및 설정
# !pip install openai  

In [4]:
# ==========================================
# [Scenario] 비밀 금고 시나리오 설정
# ==========================================
# 상황: 3개의 흩어진 단서를 조합해야 금고를 열 수 있습니다.
# 단서 1 (로그 파일): "The color is BLUE" (단순 텍스트)
# 단서 2 (암호 파일): "The shape is VQLZ" (Caesar Cipher, shift-1 -> ROUND)
# 단서 3 (외국어 문서): "The texture is SUAVE" (스페인어 -> SOFT)
#
# 목표: 최종 비밀번호 "BLUE ROUND SOFT"를 입력해야 함.
# ==========================================


In [ ]:
import os
import time
from typing import List, Dict
from openai import OpenAI

# API 키 설정 (실습 시 본인의 키를 입력하세요)
# os.environ["OPENAI_API_KEY"] = "sk-..." 
client = OpenAI()

class Vault:
    def __init__(self):
        self.clues = {
            "server_logs": "System Log: The secret color is 'BLUE'.",
            "encrypted_file": "File Content: The secret shape is 'VQLZ' (Hint: Caesar Cipher -1).",
            "foreign_document": "Note: The secret texture is 'SUAVE' (Hint: Translate Spanish to English)."
        }
        self.solution = {"BLUE", "ROUND", "SOFT"}

    def get_clues(self):
        return self.clues

    def unlock(self, code_attempt):
        # 대소문자 무시 및 단어 추출하여 비교
        attempt_set = set(code_attempt.upper().split())
        # 입력된 단어들 중 정답 단어들이 모두 포함되어 있는지 확인
        if self.solution.issubset(attempt_set):
            return True, "🔓 금고가 열렸습니다! 축하합니다!"
        else:
            return False, f"🔒 비밀번호 오류! 입력한 코드: {code_attempt}"

# ==========================================
# [Common] 기본 에이전트 클래스 정의
# ==========================================
class Agent:
    def __init__(self, name, role_prompt, model="gpt-4.1-mini"):
        self.name = name
        self.role_prompt = role_prompt
        self.model = model

    def think_and_act(self, context):
        """
        주어진 문맥(Context)을 보고 에이전트가 할 말을 생성합니다.
        """
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": self.role_prompt},
                    {"role": "user", "content": f"현재 상황(Context):\n{context}"}
                ],
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error: {str(e)}"

# 전문가 에이전트 3인방 생성
# 1. 해커: 로그 분석 전문가
hacker_prompt = """
당신은 '해커'입니다. 시스템 로그를 분석하여 숨겨진 정보를 찾는 것이 전문입니다.
주어진 텍스트에서 '색깔(Color)'과 관련된 정보를 발견하면 즉시 보고하세요.
암호나 외국어는 당신의 분야가 아니므로 무시하세요.
답변은 간결하게 핵심만 말하세요.
"""

# 2. 암호학자: 암호 해독 전문가
crypto_prompt = """
당신은 '암호학자'입니다. Caesar Cipher 같은 암호문을 해독하는 것이 전문입니다.
주어진 텍스트에서 암호화된 단어를 발견하면 해독하여 보고하세요.

중요: 'VQLZ'는 Caesar Cipher로 암호화된 단어입니다.
각 문자를 알파벳에서 앞으로(역방향) 이동하면 'ROUND'가 됩니다.
- V -> R, Q -> O, L -> U, Z -> N = ROUND

답변은 간결하게 핵심만 말하세요. 해독된 단어 하나만 말하세요.
단순 로그나 외국어는 당신의 분야가 아니므로 무시하세요.
"""

# 3. 언어학자: 번역 전문가
linguist_prompt = """
당신은 '언어학자'입니다. 외국어를 감지하고 영어로 번역하는 것이 전문입니다.
주어진 텍스트에서 외국어(예: 스페인어)를 발견하면 번역해서 보고하세요.

로그 분석이나 암호는 당신의 분야가 아니므로 무시하세요.
답변은 간결하게 핵심만 말하세요. 번역된 영어 단어 하나만 말하세요.
"""

# =============================================================
# [Experiment 1] Shared Memory (Blackboard) 아키텍처
# 특징: 리더가 없음. 게시판(Blackboard)을 보고 누구나 자율적으로 참여.
# =============================================================
def run_shared_memory_architecture():
    print("\n" + "="*50)
    print("🚀 실험 1: Shared Memory (Blackboard) 아키텍처 시작")
    print("="*50)

    vault = Vault()
    clues = vault.get_clues()
    
    # 1. 블랙보드 초기화 (모든 단서를 게시판에 붙여둠)
    blackboard = [
        f"[Clue 1] {clues['server_logs']}",
        f"[Clue 2] {clues['encrypted_file']}",
        f"[Clue 3] {clues['foreign_document']}"
    ]
    
    agents = [
        Agent("해커", hacker_prompt),
        Agent("암호학자", crypto_prompt),
        Agent("언어학자", linguist_prompt)
    ]

    print(f"📋 초기 게시판 상태:\n" + "\n".join(blackboard) + "\n")

    # 2. 자율 협업 루프 (Autonomous Loop)
    # 각 에이전트는 게시판을 보고, 자신이 기여할 수 있는 게 있으면 말합니다.
    max_turns = 1  # 간단한 예시를 위해 1턴만 진행 (실제로는 해결될 때까지 반복)
    
    for turn in range(max_turns):
        print(f"--- Turn {turn + 1} ---")
        current_context = "\n".join(blackboard)
        
        new_insights = []
        
        for agent in agents:
            print(f"🤔 {agent.name}가 게시판을 읽고 있습니다...")
            # 에이전트는 현재 게시판 전체를 보고 판단함
            response = agent.think_and_act(current_context)
            
            # 의미 있는 반응이 있으면 게시판에 추가 (실제로는 LLM 판단 로직 필요)
            # 여기서는 편의상 모든 응답을 유의미하다고 가정하고 출력
            print(f"   🗣️ {agent.name}: {response}")
            new_insights.append(f"[{agent.name}의 발견]: {response}")
        
        # 게시판 업데이트 (Memory Update)
        blackboard.extend(new_insights)

    # 3. 최종 취합 (원래는 이것도 에이전트가 해야 하지만, 실습 편의상 자동 취합)
    print("\n🔑 최종 비밀번호 조합 시도 (게시판 내용 기반)...")
    final_context = "\n".join(blackboard)
    
    # 별도의 '해결사' 에이전트가 게시판을 보고 답을 입력한다고 가정
    solver = Agent("해결사", "당신은 주어진 문맥에서 비밀번호 단어 3개를 찾아 조합하여 답을 말합니다. 형식: '단어1 단어2 단어3'")
    final_code = solver.think_and_act(final_context)
    print(f"🎯 제출된 코드: {final_code}")
    
    success, msg = vault.unlock(final_code)
    print(msg)


In [6]:

# =============================================================
# [Experiment 2] Hierarchical (Centralized) 아키텍처
# 특징: 매니저가 존재. 매니저가 계획(Plan)을 세우고 지시(Instruct)함.
# =============================================================
def run_hierarchical_architecture():
    print("\n" + "="*50)
    print("🚀 실험 2: Hierarchical (Centralized) 아키텍처 시작")
    print("="*50)

    vault = Vault()
    clues = vault.get_clues()
    
    # 1. 작업자(Worker) 에이전트 준비
    workers = {
        "hacker": Agent("해커", hacker_prompt),
        "cryptographer": Agent("암호학자", crypto_prompt),
        "linguist": Agent("언어학자", linguist_prompt)
    }

    # 2. 매니저(Manager) 에이전트 정의
    manager_prompt = """
    당신은 '프로젝트 매니저'입니다. 
    당신의 목표는 3명의 전문가들의 보고를 받아 최종 비밀번호 3개 단어를 추출하는 것입니다.
    
    [전문가들의 보고]
    - 해커: 색깔(Color) 관련 단어 추출
    - 암호학자: 암호 해독 결과
    - 언어학자: 번역된 영어 단어
    
    전문가들의 보고에서 추출된 3개의 핵심 단어를 찾아서 
    반드시 다음 형식을 준수하여 답하세요:
    비밀번호: [단어1] [단어2] [단어3]
    
    (설명이나 다른 말은 하지 말고, 오직 "비밀번호: [단어1] [단어2] [단어3]" 형식만 출력)
    """
    manager = Agent("매니저", manager_prompt)

    print("👔 매니저가 단서를 확인하고 계획을 수립합니다...")

    # 3. 매니저 주도 워크플로우 (Orchestration)
    # 매니저가 순차적으로(Sequential) 작업을 시키는 과정을 시뮬레이션
    
    # Step 1: 매니저 -> 해커
    print("\n[Step 1] 매니저 -> 해커 지시 중...")
    task1 = clues['server_logs']
    result1 = workers['hacker'].think_and_act(task1)
    print(f"   ↳ 해커 보고: {result1}")

    # Step 2: 매니저 -> 암호학자
    print("\n[Step 2] 매니저 -> 암호학자 지시 중...")
    task2 = clues['encrypted_file']
    result2 = workers['cryptographer'].think_and_act(task2)
    print(f"   ↳ 암호학자 보고: {result2}")

    # Step 3: 매니저 -> 언어학자
    print("\n[Step 3] 매니저 -> 언어학자 지시 중...")
    task3 = clues['foreign_document']
    result3 = workers['linguist'].think_and_act(task3)
    print(f"   ↳ 언어학자 보고: {result3}")

    # 4. 매니저의 최종 판단
    print("\n👔 매니저가 모든 보고를 취합하여 최종 결론을 내립니다.")
    all_reports = f"보고1: {result1}\n보고2: {result2}\n보고3: {result3}"
    final_decision = manager.think_and_act(all_reports)
    print(f"📣 매니저의 결론: {final_decision}")

    # 검증
    success, msg = vault.unlock(final_decision)
    print(msg)


In [9]:

# ==========================================
# 실행 및 비교
# ==========================================

# 1번 아키텍처 실행
run_shared_memory_architecture()


🚀 실험 1: Shared Memory (Blackboard) 아키텍처 시작
📋 초기 게시판 상태:
[Clue 1] System Log: The secret color is 'BLUE'.
[Clue 2] File Content: The secret shape is 'VQLZ' (Hint: Caesar Cipher -1).
[Clue 3] Note: The secret texture is 'SUAVE' (Hint: Translate Spanish to English).

--- Turn 1 ---
🤔 해커가 게시판을 읽고 있습니다...
   🗣️ 해커: 발견된 색깔 정보: 'BLUE'
🤔 암호학자가 게시판을 읽고 있습니다...
   🗣️ 해커: 발견된 색깔 정보: 'BLUE'
🤔 암호학자가 게시판을 읽고 있습니다...
   🗣️ 암호학자: ROUND
🤔 언어학자가 게시판을 읽고 있습니다...
   🗣️ 암호학자: ROUND
🤔 언어학자가 게시판을 읽고 있습니다...
   🗣️ 언어학자: SOFT

🔑 최종 비밀번호 조합 시도 (게시판 내용 기반)...
   🗣️ 언어학자: SOFT

🔑 최종 비밀번호 조합 시도 (게시판 내용 기반)...
🎯 제출된 코드: BLUE ROUND SOFT
🔓 금고가 열렸습니다! 축하합니다!
🎯 제출된 코드: BLUE ROUND SOFT
🔓 금고가 열렸습니다! 축하합니다!


In [10]:
# 2번 아키텍처 실행
run_hierarchical_architecture()


🚀 실험 2: Hierarchical (Centralized) 아키텍처 시작
👔 매니저가 단서를 확인하고 계획을 수립합니다...

[Step 1] 매니저 -> 해커 지시 중...
   ↳ 해커 보고: 발견된 색깔: BLUE

[Step 2] 매니저 -> 암호학자 지시 중...
   ↳ 해커 보고: 발견된 색깔: BLUE

[Step 2] 매니저 -> 암호학자 지시 중...
   ↳ 암호학자 보고: ROUND

[Step 3] 매니저 -> 언어학자 지시 중...
   ↳ 암호학자 보고: ROUND

[Step 3] 매니저 -> 언어학자 지시 중...
   ↳ 언어학자 보고: SOFT

👔 매니저가 모든 보고를 취합하여 최종 결론을 내립니다.
   ↳ 언어학자 보고: SOFT

👔 매니저가 모든 보고를 취합하여 최종 결론을 내립니다.
📣 매니저의 결론: 비밀번호: BLUE ROUND SOFT
🔓 금고가 열렸습니다! 축하합니다!
📣 매니저의 결론: 비밀번호: BLUE ROUND SOFT
🔓 금고가 열렸습니다! 축하합니다!


In [ ]:
# ==========================================
# [생각해볼 포인트]
#  
# 1. 토큰 비용: 
#    - Shared Memory 방식은 매 턴마다 모든 에이전트가 전체 게시판(Context)을 읽어야 해서 토큰 소모가 큽니다.
#    - Hierarchical 방식은 필요한 정보만 1:1로 주고받아 효율적일 수 있습니다.
#
# 2. 확장성 및 속도:
#    - Shared Memory는 에이전트가 늘어나면 게시판이 난잡해지고, 누가 언제 말할지 순서 정하기가 어렵습니다(Race Condition).
#    - Hierarchical은 매니저가 똑똑해야만 잘 돌아갑니다. 매니저가 병목(Bottleneck)이 될 수 있습니다.
#
# 3. 적합성:
#    - 아이디어 회의 같은 창의적 작업 -> Shared Memory
#    - 정해진 절차(SOP)가 있는 업무 -> Hierarchical
# ==========================================